# Example of Hidden Markov Model

In [2]:
import numpy as np
import pytensor as pt
import pymc as pm

pm.__version__, pt.__version__

('5.0.2', '2.9.1')

In [3]:
from platform import python_version

python_version()

'3.11.0'

## First understand scan function

Example from docs:

 - https://pytensor.readthedocs.io/en/latest/library/scan.html
 
 Compute `A**k`

In [13]:
k = pt.tensor.iscalar("k")
A = pt.tensor.vector("A")

# Symbolic description of the result
result, updates = pt.scan(
    fn=lambda prior_result, 
    A: prior_result * A,
    outputs_info=pt.tensor.ones_like(A),
    non_sequences=A,
    n_steps=k
)

# We only care about A**k, but scan has provided us with A**1 through A**k.
# Discard the values that we don't care about. Scan is smart enough to
# notice this and not waste memory saving them.
final_result = result[-1]

# compiled function that returns A**k
power = pt.function(inputs=[A,k], outputs=final_result, updates=updates)

print(power(range(10),2))
print(power(range(10),4))

[ 0.  1.  4.  9. 16. 25. 36. 49. 64. 81.]
[0.000e+00 1.000e+00 1.600e+01 8.100e+01 2.560e+02 6.250e+02 1.296e+03
 2.401e+03 4.096e+03 6.561e+03]


2nd example from docs: Iterating over the first dimension of a tensor: Calculating a polynomial

In [31]:
import numpy

coefficients = pt.tensor.vector("coefficients")
x = pt.tensor.scalar("x")

max_coefficients_supported = 10000

# Generate the components of the polynomial
components, updates = pt.scan(
    fn=lambda coefficient, 
    power, 
    free_variable: coefficient * (free_variable ** power),
    outputs_info=None,
    sequences=[coefficients, pt.tensor.arange(max_coefficients_supported)],
    non_sequences=x
)

# Sum them up
polynomial = components.sum()

# Compile a function
calculate_polynomial = pt.function(inputs=[coefficients, x], outputs=polynomial)

# Test
test_coefficients = numpy.asarray([1, 0, 2], dtype=numpy.float32)
test_value = 3
print(calculate_polynomial(test_coefficients, test_value))
print(1.0 * (3 ** 0) + 0.0 * (3 ** 1) + 2.0 * (3 ** 2))

19.0
19.0


Example from Medium article:
 - https://towardsdatascience.com/5-levels-of-difficulty-bayesian-gaussian-random-walk-with-pymc3-and-theano-34343911c7d2

In [38]:
def step(s, y, s_previous, α):
    s = α*y + (1-α)*s_previous
    return s

s = pt.tensor.vector('s')
y = pt.tensor.vector('y')
α = pt.tensor.scalar('α')
i = pt.tensor.scalar('i')

output, updates = pt.scan(
    fn=step,
    sequences=[s, y],
    non_sequences=[α],
    outputs_info=[i]
)

f = pt.function(
    inputs=[s, y, α, i],
    outputs=output,
    updates=updates
)

s = np.zeros(9).astype(pt.config.floatX)
y = np.arange(9).astype(pt.config.floatX)
α = 0.7
i = 0

print(f(s, y, α, i)[0:5])

[0.     0.7    1.61   2.583  3.5749]


My example. Random walk

In [25]:
noise = pt.tensor.vector("noise")



f = theano.function(inputs=[s, y, α, i],
                    outputs=output,
                    updates=updates)

# Symbolic description of the result
components, updates = pt.scan(
    fn=lambda ykm1, noise: ykm1 + noise, 
    outputs_info=None,
    sequences=[noise],
    non_sequences=y,
)

yT = components[-1]

y_last = pt.function(inputs=[noise], outputs=yT)

MissingInputError: Input 3 (y) of the graph (indices start from 0), used to compute for{cpu,scan_fn}(Subtensor{int64}.0, Subtensor{:int64:}.0, Subtensor{int64}.0, y), was not provided and not given a value. Use the PyTensor flag exception_verbosity='high', for more information on this error.
 
Backtrace when that variable is created:

  File "/Users/billtubbs/opt/anaconda3/envs/pymc/lib/python3.11/site-packages/ipykernel/zmqshell.py", line 540, in run_cell
    return super().run_cell(*args, **kwargs)
  File "/Users/billtubbs/opt/anaconda3/envs/pymc/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 2961, in run_cell
    result = self._run_cell(
  File "/Users/billtubbs/opt/anaconda3/envs/pymc/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3016, in _run_cell
    result = runner(coro)
  File "/Users/billtubbs/opt/anaconda3/envs/pymc/lib/python3.11/site-packages/IPython/core/async_helpers.py", line 129, in _pseudo_sync_runner
    coro.send(None)
  F

In [22]:
y_last(1, 10)

TypeError: Bad input argument to pytensor function with name "/var/folders/y3/28pc8qrx3dd36zwcys8z1rzc0000gn/T/ipykernel_92305/1680394650.py:15" at index 0 (0-based).  
Backtrace when that variable is created:

  File "/Users/billtubbs/opt/anaconda3/envs/pymc/lib/python3.11/site-packages/ipykernel/zmqshell.py", line 540, in run_cell
    return super().run_cell(*args, **kwargs)
  File "/Users/billtubbs/opt/anaconda3/envs/pymc/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 2961, in run_cell
    result = self._run_cell(
  File "/Users/billtubbs/opt/anaconda3/envs/pymc/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3016, in _run_cell
    result = runner(coro)
  File "/Users/billtubbs/opt/anaconda3/envs/pymc/lib/python3.11/site-packages/IPython/core/async_helpers.py", line 129, in _pseudo_sync_runner
    coro.send(None)
  File "/Users/billtubbs/opt/anaconda3/envs/pymc/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3221, in ru

Based on code from this answer by ricardoV94 on PyMC forum:
 - https://discourse.pymc.io/t/simple-markov-chain-with-aesara-scan/10745/11?u=billtubbs
 
 However, I'm using PyMC version 5 so have to replace Aesara with Pytensor.

In [6]:
import pymc as pm

k = 10

with pm.Model() as markov_chain:

    transition_probs = pm.Uniform('transition_probs', lower=0, upper=1, shape = 2)
    initial_state = pm.Bernoulli('initial_state', p = 0.5)

    def transition(previous_state, transition_probs, old_rng):
        p = transition_probs[previous_state]
        next_rng, next_state = pm.Bernoulli.dist(p = p, rng=old_rng).owner.outputs
        return next_state, {old_rng: next_rng}

    rng = pt.shared(np.random.default_rng())
    mc_chain, updates = pt.scan(fn=transition,
                                  outputs_info=dict(initial = initial_state),
                                  non_sequences=[transition_probs, rng],
                                  n_steps=k)
    assert updates
    markov_chain.register_rv(mc_chain, name="mc_chain", initval="prior")

with markov_chain:
    pm.sample(chains=1, step=pm.BinaryMetropolis([mc_chain]))

ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: transform_scan_values
ERROR (pytensor.graph.rewriting.basic): node: for{cpu,scan_fn}(TensorConstant{10}, IncSubtensor{Set;:int64:}.0, RandomGeneratorSharedVariable(<Generator(PCG64) at 0x16A756F80>), transition_probs)
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/Users/billtubbs/opt/anaconda3/envs/pymc/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1925, in process_node
    replacements = node_rewriter.transform(fgraph, node)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/billtubbs/opt/anaconda3/envs/pymc/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1084, in transform
    return self.fn(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/billtubbs/opt/anaconda3/envs/pymc/lib/python3.11/site-packages/pymc/logprob/transforms.py", line 239, in transform_sca

Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 8 seconds.
/Users/billtubbs/opt/anaconda3/envs/pymc/lib/python3.11/site-packages/arviz/data/base.py:221: UserWarning: More chains (1000) than draws (1). Passed array should have shape (chains, draws, *shape)
  warnings.warn(


In [7]:
with markov_chain:
    trace = pm.sample_prior_predictive(1000, compile_kwargs=dict(updates=updates))

Sampling: [initial_state, mc_chain, transition_probs]


In [8]:
trace

Inference data with groups:
	> prior

## Example from stackoverflow answer

 - https://stackoverflow.com/a/29256808/1609514

In [39]:
from IPython.core.debugger import set_trace

def generate_timesteps(N, p_init, p_trans):

    set_trace()
    timesteps = np.empty(N+1, dtype=object)

    # A success denotes being in state 2, a failure being in state 1
    timesteps[0] = pm.Bernoulli('T0', p_init)

    for i in range(1, N):
        # probability of being in state 1 at time step `i` given time step `i-1`
        p_i = p_trans[1] * timesteps[i-1] + p_trans[0] * (1-timesteps[i-1])
        timesteps[i] = pm.Bernoulli('T%d' % i, p_i)

    return timesteps

In [38]:
with pm.Model() as markov_chain:
    timesteps = generate_timesteps(10, 0.8, [0.001, 0.5])
    model = pm.MCMC(timesteps)
    model.sample(10000) # no burn in necessary since we're sampling directly from the distribution
    [np.mean(model.trace(t).gettrace()) for t in timesteps]

# THIS DOESN'T WORK

> /var/folders/y3/28pc8qrx3dd36zwcys8z1rzc0000gn/T/ipykernel_85892/1522797367.py(6)generate_timesteps()
      4 
      5     set_trace()
----> 6     timesteps = np.empty(N+1, dtype=object)
      7 
      8     # A success denotes being in state 2, a failure being in state 1

ipdb> c


AttributeError: module 'pymc' has no attribute 'MCMC'